<a href="https://colab.research.google.com/github/phi1z/1yanagiLab/blob/main/Nanocage/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Requestments { display-mode: "form" }

# @markdown ## <font color="OrangeRed"> ← **First, Please Run This Cell!** </font>

!pip install tqdm
!pip install ipywidgets==7.7.1

# install modules and set its path
import os
import sys
from tqdm.notebook import tqdm
import shutil
import re
import ipywidgets as widgets
from IPython.display import display

# make basic dirs
os.makedirs('Libraries', exist_ok=True)
os.makedirs('Data', exist_ok=True)

# append package path
sys.path.append('/content/Libraries')

# install packages
#!pip install -t Libraries/ numpy

# import required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
import scipy.stats
import scipy.signal
from scipy.spatial.transform import Rotation
from datetime import datetime

from IPython.display import HTML
from IPython.display import display, clear_output

clear_output(wait=True)

print("All modules have been successfully installed!")

# Method

## Model
We set four conditions:
1. Collagen shrinks like a spring.
2. Nanocages don't leave the collagen.
3. Nanocages are arranged in the same interval.
4. Outer AMF affects all nanocages in the same way.

## Equations
The equation of motion (EOM) of the $i$-th nanocage (total $n$) can be written regarding spring forces from both sides of nanocages with $i$-th nanocage's phase space ($\vec{r}=(x,y,z)^\top$ and $\vec{v} = (v_x, v_y,v_z)^\top$),

$$m\ddot{\vec{r}}_i = -k\Delta l_i\frac{\vec{r}_{i-1} - \vec{r}_i}{|\vec{r}_{i-1} - \vec{r}_i|} + k\Delta l_{i+1}\frac{\vec{r}_{i+1} - \vec{r}_{i}}{|\vec{r}_{i+1} - \vec{r}_{i}|}  + \vec{F}(t) $$

$$\quad ⟺ \begin{cases} \frac{d}{dt} \vec{r}_i = \vec{v}_i \\
\frac{d}{dt} \vec{v}_i = \frac{k}{m}\left(-\Delta l_i\vec{e}_{i, i-1} + \Delta l_{i+1}\vec{e}_{i, i+1} \right) + \frac{1}{m} \vec{F}(\vec{r}, \vec{\dot{r}}, t) \end{cases} \\
\vec{e}_{i,j} = \frac{\vec{r}_j - \vec{r}_i}{|\vec{r}_j - \vec{r}_i|}, \quad
\Delta l_i = |\vec{r}_{i-1} - \vec{r}_i| - l_{0i}
$$
where $\vec{e}_{i,j}$ is tha unit vector from $i$-th nanocage to $j$-th one, $\Delta l_i$ is the change in length of collagen between $(i-1)$-th nanocage to $i$-th one, $m$ is the mass of the nanocage (we assume all masses are the same), $k$ is the spring constant of collagen, and $F$ is the outer force, such as magnetic or Brownian.

The boundary condition is represent following equations,

$$\Delta l_0=0,\: \Delta l_{n+1}=0$$

The initial condition (or the case in first time ($t=0$)) are:

$$ l_{0i} = l_i(t=0) = \textrm{random} \:( 0 < l_{0i} < l_\textrm{0-max} ) \\
\theta_{0i} = \theta_i(t=0) = \textrm{random} \:( |\theta_{0i}| < \theta_\textrm{0-max} ) \\
\phi_{0i} = \phi_i(t=0) = \textrm{random} \:( |\phi_{0i}| < \phi_\textrm{0-max} ) \\
\vec{v}(t=0) = \vec{0}$$

where $\theta_i, \phi_i$ is the angle between $(i-1)$-th nanocage to $i$-th one.
Of course, $\vec{r}_1$ and $\vec{r}_{n}$ follow boundary conditions in preference.

## Outer Force

### AC Magnetic Field

Nanocage exhibits magnetization and responds to the force from an AC magnetic field (AMF).
However, we don't consider susceptibility $\chi^{\rm (AC)}$, in other words, nanocages magnetize immediately.

$$\vec{F}_{\rm AMF}(t) = \vec{A}\sin(2 \pi f_{\rm AMF} t)$$

where $\vec{A}$ (unit $\rm N$) is the magnitude of AMF and $f_{\rm AMF}$ (unit $\rm Hz$) is the frequency of AMF.

### Viscous Resistance

The resistive force exerted on an object moving through a fluid is proportional to its velocity and can be expressed using a constant $b$ as follows.

$$\vec{F}_{{\rm VR}, i}(\vec{\dot{r}}, t) = -b\vec{\dot{r}}_i $$

### Brownian motion

Brownian motion can be described as a random walk.
A random walk refers to movement in a random direction with a constant magnitude at each time interval $\Delta t$ (corresponding to one frame).
In this study, the Brownian motion term was not added to the external force. However, the displacement caused by Brownian motion influences the calculation of other forces. Specifically, a random walk term is added to the time derivative of the position $\dot{r}$.

$$\frac{d}{dt} \vec{r}_i = \vec{v}_i + \frac{\lambda}{\Delta t}\vec{\epsilon} $$

where, $\vec{\epsilon}$ is a random directional vector with a magnitude less than 1, and $\lambda$ represents the maximum distance that the particle can move in a single frame.

## Discrete Fourier transform (DFT)
The discrete Fourier transform (DFT) was implemented to analyze the oscillatory (or wave) components present in the motion.
First, to obtain the displacement of the nanocage, we define the phase $\Delta \vec{r}_i(t)$ for the i-th coordinate $\vec{r}_i(t)$ as follows:

$$\Delta \vec{r}_i(t) = \vec{r}_i(t) - \vec{r}_i(t=0)$$

If the `Displacement` option is unchecked, the DFT will be performed on the coordinate $\vec{r}_i(t)$ itself rather than on the phase.
However, because the absolute coordinates are arbitrary, disabling this option is not recommended.

The DFT is performed using the Python function `numpy.fft.fft()`. Briefly, for a phase signal of length $M$ (equal to the number of frames), the Fourier-transformed output is defined as follows:

$$ {\mathscr F}[\Delta \vec{r}_i(t)] = \vec{F}(f) = \sum^{M-1}_{s=0} \Delta \vec{r}_i\left(s\frac{t_{\rm end}}{M}\right) \exp(-i\omega s), \qquad \left( \omega = \frac{2\pi f}{M} \right)$$

Because the FFT algorithm is most efficient when $M$ is a power of two, it is strongly recommended that the number of frames be set to $2^n$.
The DFT corresponds to decomposing the motion into a series of simple harmonic components. A larger value of $\vec{F}(f)$ indicates that a simple harmonic oscillation with frequency $f$ contributes more strongly to the motion $\Delta \vec{r}(t)$ of the nanocage.

Because $\vec{F}(f)$ is a complex number, its real part corresponds to $\cos(x)$ and its imaginary part corresponds to $\sin(x)$; however, this distinction merely reflects a phase difference and is not essential. Therefore, the absolute values of $|\vec{F}(f)|$ are taken and consider only its amplitude.

$$ |\vec{F}(f)| = \sqrt{{\rm Re}[\vec{F}(f)]^2 + {\rm Im}[\vec{F}(f)]^2} $$

##Monte-Carlo method

Most motions exhibit chaotic behavior; that is, even slight differences in initial conditions can lead to completely different trajectories, making deterministic prediction impossible. However, such motions are not entirely unpredictable, and their statistical tendencies can still be analyzed.
To statistically characterize the motion of the nanocages, we employ the Monte-Carlo method.

First, N simulations are performed under identical conditions except for the initial position of the nanocage. The initial condition corresponds to the position of the nanocage and reflects random deformation and tension applied to the collagen.
For each simulated trajectory, a DFT is conducted, yielding a total of $3N$ Fourier transforms $F(f)$.
For each direction $(x, y, z)$, $N$ $F_{x,y,z}(f)$s are computed the mean to obtain the statistical DFT, and the error was defined as the standard error.

In addition, the mean of radial DFT $\bar{F}_r(f)$, which accounts for all spatial directions, is calculated as follows.

$$ \bar{F}_r(f) = \sqrt{\bar{F}_x(f)^2 + \bar{F}_y(f)^2 + \bar{F}_z(f)^2} $$

# 3D Motion

In [ ]:
# @title Set Conditions { run: "auto", display-mode: "form"}

#@markdown ### Scale and world setting

#@markdown scale of basic unit [log]
Scale_time = -9 # @param {"type":"integer"}
Scale_mass = -21 # @param {"type":"integer"}
Scale_length = -6 # @param {"type":"integer"}
Scales = np.array([Scale_time, Scale_mass, Scale_length])

def scale_dimention(T=0, M=0, L=0):
  pow = np.dot(np.array([T, M, L]), Scales )
  return np.power(10.0, pow)

# Stage
#@markdown time span of simulation [xs]
t_end = 20 # @param {"type":"number"}
t_span = np.array([0, t_end]) # start, end [s]
t_span = t_span * scale_dimention(T=1)
#@markdown number of simulation frame (please set $2^n$ to calculate FFT easily!)
frames = 8192 # @param {"type":"integer"}
t_eval = np.linspace(t_span[0], t_span[1], frames)

#@markdown ### Collagen and Nanocages

#@markdown the number of nanocage: $n$ [-]
n = 12 # @param {"type":"integer"}
#@markdown the distance between cages: $x_s$ [xm]
x_distans_per = 0.5 # @param {"type":"number"}
x_distans_per = x_distans_per * scale_dimention(L=1)
#@markdown the maximum initial distance of cages: $l_{0 \rm max}$ [xm]
distance_0max = 0.05 # @param {"type":"number"}
distance_0max = distance_0max * scale_dimention(L=1)
#@markdown the maximum angle between nanocage and next one : $\theta_{0 \rm max}, \phi_{0 \rm max}$ [deg.]
theta_0max = 10 # @param {"type":"number"}
phi_0max = 10 # @param {"type":"number"}
theta_0max = theta_0max * np.pi / 180 # the maximum initial angle: \theta_{0-max} [rad]
phi_0max = phi_0max * np.pi / 180 # the maximum initial angle: \theta_{0-max} [rad]
length_collagen = n * x_distans_per # the length of collagen [m]

#@markdown ### Physical Constants

# Constants
#@markdown mass of a nanocage: $m$ [xkg]
Mass = 20  # @param {"type":"number"}
Mass = Mass * scale_dimention(M=1)
#@markdown spring constant of collagen: $k$ [N/m]
# k = 144 # from KAKENHI-PROJECT-22700474/22700474
k = 0.5 # @param {"type":"number"}
res_freq = np.sqrt(k/Mass) / (2*np.pi)

#@markdown ### Outer Force

#@markdown Amplitude of AMF: $\vec{A}$ [xN (xkg$\cdot$xm/xs$^2$)]
A = [0, 0, 0] # @param
A = np.array(A) * scale_dimention(M=1, T=-2, L=1)
#@markdown Frequency of AMF: $f$ [xHz (1/xs)]
f = 1 # @param {"type":"number"}
f = f * scale_dimention(T=-1)
#@markdown Constant of viscous resistance: $b$ [xkg/xs]
b = 1 # @param {"type":"number"}
b = b * scale_dimention(M=1, T=-1)

#@markdown Brownian motion: $\lambda$ [xm]
c = 0 # @param {"type":"number"}
c = c * scale_dimention(L=1)
c = c / (t_span[1] / frames)

condition_table = pd.DataFrame()
condition_table["Time_start"] = [t_span[0]]
condition_table["Time_end"] = [t_span[1]]
condition_table["Frame"] = frames
condition_table["Number_of_nanocages"] = [n]
condition_table["Mass_of_nanocage"] = [Mass]
condition_table["Distance_between_nanocages"] = [x_distans_per]
condition_table["Maximum_initial_distance"] = [distance_0max]
condition_table["Maximum_initial_theta"] = [theta_0max]
condition_table["Maximum_initial_phi"] = [phi_0max]
condition_table["Spring_constant_of_collagen"] = [k]
condition_table["AMF_Amplitude"] = [A]
condition_table["AMF_Frequency"] = [f]
condition_table["Viscous_resistance"] = [b]
condition_table["Brownian_motion"] = [c]

condition_text = ""
condition_text += f"Time: {t_span[0]:.3g} s---{t_span[1]:.3g} s\n"
condition_text += f"Time Resolution: {t_span[1] / frames * 1.0e9:.3g} ns\n"
condition_text += f"Number of nanocages: {n}\n"
condition_text += f"Mass of nanocage: {Mass * 1e3:.3g} g\n"
condition_text += f"Distance between nanocages: {x_distans_per:.3g} m\n"
condition_text += f"Maximum initial distance: {distance_0max:.3g} m\n"
condition_text += f"Maximum initial angle: {theta_0max:.3g} rad, {phi_0max:.3g} rad\n"
condition_text += f"Spring constant of collagen: {k:.3g} N/m\n"
condition_text += f"Resonant frequency: {res_freq * 1.0e-6:.3g} MHz\n"
condition_text += f"AMF: ({A[0]:.3g}, {A[1]:.3g}, {A[2]:.3g}) N, {f:.3g} Hz\n"
condition_text += f"Viscous resistance: {b:.3g} kg/s\n"
condition_text += f"Brownian motion constant: {c:.3g} m/s\n"

#transpose table
condition_table = condition_table.transpose()
#output as csv
condition_table.to_csv("condition.csv")

print("="*15 + " Condition " +"="*15)
print(condition_text)
print("="*17 + " Save " +"="*17)
print(f"Save these conditions as [condition.csv]")


In [ ]:
# @title Simulation: 3D motion

thetas_initial = np.random.uniform(-1, 1, size=n-1) * theta_0max
for i in range(1, n-1, 1):
  thetas_initial[i] = thetas_initial[i] + thetas_initial[i-1]

l_natural = np.ones(n-1) * x_distans_per
l_initial =  l_natural + (np.random.uniform(-1, 1, size=n-1) * distance_0max)

# EOM const.
Spring_parameter = k / Mass
def FVR(v, Bcont):
  F = -1 * Bcont * v
  return F

def Brownian_motion(amp, n):
  vec = np.random.rand(3*n) - 0.5
  vec = vec / 0.866
  return amp * vec

def length(xv, i):
  this_vec = np.array([xv[i], xv[i+n], xv[i+2*n]])
  next_vec = np.array([xv[i+1], xv[i+n+1], xv[i+2*n+1]])
  return np.linalg.norm(next_vec - this_vec)

def Dlength(xv, i, l0):
  return length(xv, i) - l0

def unit_vec(xv, i):
  this_vec = np.array([xv[i], xv[i+n], xv[i+2*n]])
  next_vec = np.array([xv[i+1], xv[i+n+1], xv[i+2*n+1]])
  return (next_vec - this_vec) / np.linalg.norm(next_vec - this_vec)

def AMF(t, freq, Amp):
  return Amp * np.cos(2 * np.pi * freq * t)

def initial_condition3D(n):
  l_natural = np.ones(n-1) * x_distans_per
  l_initial =  l_natural + (np.random.uniform(-1, 1, size=n-1) * distance_0max)
  thetas_initial = np.random.uniform(-1, 1, size=n-1) * theta_0max
  phis_initial = np.random.uniform(-1, 1, size=n-1) * phi_0max
  xv = np.zeros(6*n) # [r_1, r_2,...,r_n(x,y,z), v_1, v_2,...,vx_n(xyz)]
  for i in range(1, n, 1):
    unit_collagen_vec = np.array([1, 0, 0])
    unit_collagen_vec = l_initial[i-1] * np.array([1, 0, 0])
    rot_mat = Rotation.from_rotvec(np.array([0, thetas_initial[i-1], phis_initial[i-1]])).as_matrix()
    xv[[i, i+n, i+2*n]] = xv[[i-1, i+n-1, i+2*n-1]] + np.dot(unit_collagen_vec, rot_mat)
  return xv

xv = initial_condition3D(n)

def EOM(t, x_v):
  # velocity
  next = np.zeros_like(x_v)
  next[0:(3*n)] = x_v[(3*n):(6*n)]
  next[0:(3*n)] = next[0:(3*n)] + Brownian_motion(c, n) # Brownian motion
  # i=1
  T_initial = Dlength(x_v, 0, l_natural[0]) * Spring_parameter
  e_initial = unit_vec(x_v, 0)
  next[[3*n, 4*n, 5*n]] = T_initial * e_initial
  # 1 < i < n
  for i in range(1 ,n-1, 1):
    T_prev = Dlength(x_v, i-1, l_natural[i-1]) * Spring_parameter
    e_prev = unit_vec(x_v, i-1)
    T_next = Dlength(x_v, i, l_natural[i]) * Spring_parameter
    e_next = unit_vec(x_v, i)
    next[[3*n+i, 4*n+i, 5*n+i]] = -1 * T_prev * e_prev + T_next * e_next
  # i = n
  T_final = Dlength(x_v, n-2, l_natural[n-2]) * Spring_parameter
  e_final = unit_vec(x_v, n-2)
  next[[4*n-1, 5*n-1, 6*n-1]] = -1 * T_final * e_final
  # outer force
  F = np.zeros(3*n)
  F = F + np.repeat(AMF(t, freq=f, Amp=A), n) #AMF
  F = F + FVR(x_v[(3*n):(6*n)], b) #VR
  F = F / Mass
  next[3*n:(6*n)] = next[3*n:(6*n)] + F
  return next

sol = scipy.integrate.solve_ivp(EOM, t_span, xv, t_eval=t_eval)

def plot_position(i, mode="xy"):
  x_eval = sol.y[i]
  y_eval = sol.y[n+i]
  z_eval = sol.y[2*n+i]
  if mode == "xy":
    plt.plot(x_eval, y_eval, label=f"{i+1}")
  elif mode == "zy":
    plt.plot(z_eval, y_eval, label=f"{i+1}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.title("x-y motion")
for i in range(0, n, 1):
  plot_position(i)
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.legend()

plt.subplot(1, 2, 2)
plt.title("y-z motion")
for i in range(0, n, 1):
  plot_position(i, mode="zy")
plt.xlabel('z [m]')
plt.ylabel('y [m]')
plt.legend()

# plt.savefig("position.png", dpi=300)
plt.show()

In [ ]:
#@title Make Animation
from IPython.display import HTML
from matplotlib.animation import ArtistAnimation

def max_arrays(*arrays):
  if not arrays:
    return None
  combined_array = np.concatenate(arrays)
  return np.max(combined_array)

def min_arrays(*arrays):
  if not arrays:
    return None
  combined_array = np.concatenate(arrays)
  return np.min(combined_array)

time_scale = float(np.pow(10, -1 * Scale_time))
lim_x = np.array([min_arrays(sol.y[0:(n)]), max_arrays(sol.y[0:(n)])])
lim_y = np.array([min_arrays(sol.y[n:(2*n)]), max_arrays(sol.y[n:(2*n)])])
lim_z = np.array([min_arrays(sol.y[(2*n):(3*n)]), max_arrays(sol.y[(2*n):(3*n)])]) # Get z limits

#@markdown Set the range to export as an animation [%]
Animation_range = 100 # @param {"type":"slider","min":0,"max":100,"step":1}
Animation_range = float(Animation_range) / 100
Animation_frame = int(round(frames * Animation_range))
print(f"Range of animation: {Animation_range*100:.0f} %")

fig, ax = plt.subplots(figsize = (2*n, 4))
# Set plot limits based on overall min/max for a consistent scale
ax.set_xlim(lim_x[0]*1.1, lim_x[1]*1.1)
ax.set_ylim(lim_y[0]*1.1, lim_y[1]*1.1)
ax.set_aspect('equal')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
FRAMES = []

#@markdown Check and you can change FPS (**Recommend checking it out!**)
Set_FPS = True # @param {"type":"boolean"}
#@markdown Set the FPS of Animation.mp4
Animation_FPS = 24 # @param {"type":"slider","min":1,"max":100,"step":1}

#@markdown Set color gradiation of z-axis (see the color sets [OFFICIAL DOCUMENT](https://matplotlib.org/stable/users/explain/colors/colormaps.html))
colormap_name = "gist_gray" # @param ["coolwarm_r","gist_gray","viridis_r","gist_rainbow","plasma_r"] {"allow-input":true}
colormap = getattr(plt.cm, colormap_name)

if Set_FPS:
  MP4 = []
  interval = frames / (t_span[1] * scale_dimention(T=-1) * Animation_FPS)
  if interval < 1:
    raise ValueError("Animation frames are too big. Please decreace FPS.")
  interval = int(round(interval))
  Animation_frame = np.arange(0, Animation_frame, interval)
else:
  Animation_frame = range(Animation_frame)

print(f"Number of animation frame: {len(Animation_frame)}")
print("="*40)
print(f"Estimated run time: {0.236 * len(Animation_frame) * (1.1 ** n):.1f} s")
print("="*40)

for i in Animation_frame:
  positions = np.zeros((3, n)) # xyz-position at a frame
  for j in range(n):
    positions[0, j] = sol.y[j][i] # Get x position
    positions[1, j] = sol.y[n+j][i] # Get y position
    positions[2, j] = sol.y[2*n+j][i] # Get z position

  norm_z = (positions[2, :] - lim_z[0]) / (lim_z[1] - lim_z[0]) # Normalize z position
  this_color = colormap(norm_z)

  frame_artists = [] # List to hold artists for this frame

  # Plot the line connecting the points
  line_artist = ax.plot(positions[0, :], positions[1, :], '-', linewidth=3, color="black")[0]
  frame_artists.append(line_artist)

  # Plot the points
  for j in range(n):
    point_artist = ax.plot(positions[0, j], positions[1, j], 's', markersize=10, color=this_color[j])[0] # Using a fixed color for now
    frame_artists.append(point_artist)

  # Add the time text artist
  text_time = ax.text(0.70, 0.90, r'$t = {:.2f}\times 10^{{-{}}}$ s'.format(sol.t[i] * time_scale, -1 * Scale_time),
                      fontsize=14, ha='left', va='top',
                      transform=ax.transAxes)
  frame_artists.append(text_time)

  FRAMES.append(frame_artists) # Append the list of artists for this frame

plt.close(fig)

print("plt section is done.")

if Set_FPS:
  ANIMATION = ArtistAnimation(fig, FRAMES, interval=1000 / Animation_FPS, blit=True)
else:
  ANIMATION = ArtistAnimation(fig, FRAMES, interval=1000 / (frames / t_span[1] * 10**Scale_time), blit=True)

print("animation section is done.")
if Set_FPS:
  ANIMATION.save("Animation.mp4", writer='ffmpeg', fps=Animation_FPS)
else:
  ANIMATION.save("Animation.mp4", writer='ffmpeg', fps=(frames // t_span[1] * 10**Scale_time))
print("mp4 section is done.")

# View_HTML = True # @param {"type":"boolean"}
# if View_HTML:
#   HTML(ANIMATION.to_html5_video())
HTML(ANIMATION.to_jshtml())

# Monte-Carlo simulation

In [ ]:
#@title $N\times$ simulation {display-mode: "form"}

def format_bytes(size):
    power = 1024
    n = 0
    power_labels = {0: '', 1: 'KB', 2: 'MB', 3: 'GB', 4: 'TB'}
    while size > power and n <= len(power_labels):
        size /= power
        n += 1
    return f"{size:.2f} {power_labels[n]}"

#@markdown Set the number of simulation
N = 10 # @param {"type":"integer"}
#@markdown Set the label of output file (Ex. `label_100x8cages.csv`  $N=100, n=8$)

Label = "" # @param {"type":"string"}
#@markdown **If blank, the time (HH-MM-SS) will be the label**

#@markdown Check: DTF on displacement $\Delta\vec{r} = \vec{r} - \vec{r}(t=0)$
Displacement = True # @param {"type":"boolean"}

if Label == "":
  now = datetime.now()
  Label = now.strftime("%H-%M-%S")

print(f"Simulate {N} times")
print(f"Number of nanocages: {n}")
print(f"Displacement: {Displacement}")
print(f"Table size: {frames} columns x {3*n*N+1} rows")
print("="*40)

# Create a list of column names
column_names = ["Time"]
for i in range(N):
  for j in range(n):
      column_names.append(f"{i+1}_x_{j+1}")
      column_names.append(f"{i+1}_y_{j+1}")
      column_names.append(f"{i+1}_z_{j+1}")

All_motion_table = pd.DataFrame(np.zeros((frames, 3*n*N+1)))
All_motion_table.columns = column_names

is_first = True
for i in tqdm(range(N)):
  if is_first:
    All_motion_table.iloc[:, 0] = sol.t
    is_first = False
  xv = initial_condition3D(n)
  sol = scipy.integrate.solve_ivp(EOM, t_span, xv, t_eval=t_eval)
  for j in range(n):
    x_motion = sol.y[j]
    y_motion = sol.y[n+j]
    z_motion = sol.y[2*n+j]
    if Displacement:
      x_motion = x_motion - x_motion[0]
      y_motion = y_motion - y_motion[0]
      z_motion = z_motion - z_motion[0]
    All_motion_table.iloc[:, 1 + i*3*n + j*3 + 0] = x_motion # x positions
    All_motion_table.iloc[:, 1 + i*3*n + j*3 + 1] = y_motion # y positions
    All_motion_table.iloc[:, 1 + i*3*n + j*3 + 2] = z_motion # z positions

All_motion_table.head()

# include date
dir_name = "N-simulations"
file_name = f"{Label}_{N}x{n}cages.csv"
file_path = os.path.join(dir_name, file_name)

os.makedirs(dir_name, exist_ok=True)

print(f"Saving the result......")

All_motion_table.to_csv(file_path, index=False, float_format='%.6g')

print("="*40)
print(f"Save as {file_path}")
file_size_bytes = os.path.getsize(file_path)
print(f"File size: {format_bytes(file_size_bytes)}")


In [ ]:
#@title DFT for **All** results { display-mode: "form"}
files = os.listdir("N-simulations")
result_files = [f for f in files if os.path.splitext(f)[1] == '.csv']
result_files = np.sort(result_files)

print(f"Number of result files: {len(result_files)}")

def i_dft(df, i):
  signal_x = df.iloc[:, i+1].to_numpy()
  signal_y = df.iloc[:, i+2].to_numpy()
  signal_z = df.iloc[:, i+3].to_numpy()
  n_sample = len(signal_x)
  DFT_x = np.fft.fft(signal_x)
  DFT_y = np.fft.fft(signal_y)
  DFT_z = np.fft.fft(signal_z)

  DFT_x = DFT_x[:(n_sample//2)] / n_sample * 2
  DFT_y = DFT_y[:(n_sample//2)] / n_sample * 2
  DFT_z = DFT_z[:(n_sample//2)] / n_sample * 2
  DFT = np.array([DFT_x, DFT_y, DFT_z])
  DFT = np.abs(DFT)
  return DFT

def N_dft(result, n=None):
  if n == None:
    file_name = os.path.basename(result)
    match = re.search(r'x(\d+)cages', file_name)
    if match:
      n = int(match.group(1))
    else:
      print("The number of nanocages is not found from file name.\nPlease set argument n.")
      return None
  df = pd.read_csv(result)
  N = df.shape[1] // (n*3)
  DFT_x, DFT_y, DFT_z = [], [], []
  for j in range(n):
    DFT_ix, DFT_iy, DFT_iz = [], [], []
    for i in range(N):
      DFT = i_dft(df, i*3*n + j*3)
      DFT_ix.append(DFT[0])
      DFT_iy.append(DFT[1])
      DFT_iz.append(DFT[2])
    DFT_x.append(DFT_ix)
    DFT_y.append(DFT_iy)
    DFT_z.append(DFT_iz)
  return np.array([DFT_x, DFT_y, DFT_z])

def read_time(path):
  df = pd.read_csv(path)
  time = df.iloc[:, 0].to_numpy()
  time_span = time[-1] - time[0]
  return time_span

def stat_dft(dft, time):
  dim = dft.shape
  n_sample = dim[3] * 2
  f_sample = time / n_sample
  freq = np.fft.fftfreq(n_sample, d=f_sample)
  freq = freq[0:(n_sample//2)]
  out_table = pd.DataFrame(np.zeros((len(freq), 1 + dim[1]*6)))
  column_name_base = ["mean_x_", "std_x_", "mean_y_", "std_y_", "mean_z_", "std_z_"]
  column_names = ["freq"]
  for j in range(dim[1]):
    this_column_name = [s + str(j+1) for s in column_name_base]
    column_names = column_names + this_column_name
  out_table.columns = column_names
  out_table["freq"] = freq
  for j in range(dim[1]):
    this_dft = dft[:, j, :]
    out_table[f"mean_x_{j+1}"] = np.mean(this_dft[0], axis=0)
    out_table[f"std_x_{j+1}"] = np.std(this_dft[0], axis=0, ddof=1)
    out_table[f"mean_y_{j+1}"] = np.mean(this_dft[1], axis=0)
    out_table[f"std_y_{j+1}"] = np.std(this_dft[1], axis=0, ddof=1)
    out_table[f"mean_z_{j+1}"] = np.mean(this_dft[2], axis=0)
    out_table[f"std_z_{j+1}"] = np.std(this_dft[2], axis=0, ddof=1)
  return out_table

stat_dir = "Statistic"
dft_dir = "DFT-simulations"
os.makedirs(stat_dir, exist_ok=True)
os.makedirs(dft_dir, exist_ok=True)

for this_result_file in tqdm(result_files):
  this_path = os.path.join("N-simulations", this_result_file)
  DFT_table = N_dft(this_path)
  DFT_stat = stat_dft(DFT_table, read_time(this_path))
  DFT_stat.head()

  DFT_stat_path = os.path.join(stat_dir, f"{os.path.splitext(this_result_file)[0]}_stat.csv")
  DFT_stat.to_csv(DFT_stat_path, index=False, float_format='%.6g')
  DFT_table_path = os.path.join(dft_dir, f"{os.path.splitext(this_result_file)[0]}_DFT.npy")
  np.save(DFT_table_path, DFT_table)

  print(f"Save as DFT: {DFT_table_path}")
  print(f"Save as Statistic: {DFT_stat_path}")
  file_size_bytes_dft = os.path.getsize(DFT_table_path)
  print(f"File size DFT: {format_bytes(file_size_bytes_dft)}")
  file_size_bytes_stat = os.path.getsize(DFT_stat_path)
  print(f"File size Statistic: {format_bytes(file_size_bytes_stat)}")
  print("="*40)

clear_output(wait=True)

print(f"Number of result files: {len(result_files)}")
for i, this_result_file in enumerate(result_files):
  base_name = os.path.splitext(this_result_file)[0]
  print(f"{i+1} | DFT for {base_name}")


In [ ]:
#@title Select result for Plot (**Run only the first time**) { display-mode: "form"}
files = os.listdir("Statistic")
stat_files = [f for f in files if os.path.splitext(f)[1] == '.csv']
stat_files = np.sort(stat_files)

print(f"Number of statistic files: {len(stat_files)}")

# Drop down list
this_stat_file = widgets.Dropdown(
    options=stat_files,
    value=stat_files[0],
    description='Select result:',
    disabled=False,
    layout={'width': '50%', 'height': '40px'},
    style={'description_width': 'initial'},
)

display(this_stat_file)

In [ ]:
#@title plot DFT { display-mode: "form"}
# @markdown The direction of the coordinate ($r = \sqrt{x^2 + y^2 + z^2}$)
direction = "r" # @param ["r","x","y","z"]
# @markdown The maximum range of frequency [%]
freq_range = 30 # @param {"type":"slider","min":0,"max":100,"step":1}
# @markdown $i$-th nanocages list that you want to see the **graph** (0 means all, Ex. [1,2,5])
i_list = 0 # @param
if isinstance(i_list, int):
  i_list = [i_list]
i_list = np.array(i_list)
# @markdown Plot peak on the graph
Plot_peak = True # @param {"type":"boolean"}
# @markdown $j$-th nanocages list that you want to see the **peak** (0 means all, Ex. [1,2,5])
j_list = 0 # @param
if isinstance(j_list, int):
  j_list = [j_list]
j_list = np.array(j_list)
# @markdown Set error as Standard Error (False: standard deviation)
SE = True # @param {"type":"boolean"}
if SE:
  SEN = re.search(r'(\d+)x', this_stat_file.value)[1]
  SEN = int(SEN)
  print(f"Number of simulation as N = {SEN}")
else:
  SEN = None
# @markdown The color list of lines
color_map = "viridis_r" # @param ["coolwarm_r","twilight_shifted","viridis_r","gist_rainbow","plasma_r"] {"allow-input":true}

def set_xlim_df(df, flim=25):
  n_lim = (df.shape[0] * flim) // 100
  X = df["freq"]
  X = X[1:n_lim]
  plt.xlim(X.min(), X.max())

def extract_direc(df, i, direction="x"):
  if direction == "r":
    df[f'mean_r_{i+1}'] = np.sqrt(df[f'mean_x_{i+1}']**2 + df[f'mean_y_{i+1}']**2 + df[f'mean_z_{i+1}']**2)
    df[f'std_r_{i+1}'] = np.sqrt(df[f'std_x_{i+1}']**2 + df[f'std_y_{i+1}']**2 + df[f'std_z_{i+1}']**2)
  Y = df[f"mean_{direction}_{i+1}"]
  Y_err = df[f"std_{direction}_{i+1}"]
  return Y, Y_err

def plot_peak(df, flim=25, vec="x", i=0, c=0, plot=False):
  n_lim = (df.shape[0] * flim) // 100
  if 0 in j_list:
    n = re.search(r'(\d+)cages', this_stat_file.value)[1]
    n = int(n)
    n_list = range(n)
  else:
    n_list = [int(j - 1) for j in i_list]
  cmap = plt.colormaps.get_cmap(color_map)
  if len(n_list) == 1:
    color_list = ["black"]
  else:
    color_list = [cmap(i/np.max(n_list)) for i in n_list]
  X = df["freq"]
  X = X[1:n_lim]
  Y, _ = extract_direc(df, i, direction=vec)
  Y = Y[1:n_lim]
  Y = np.log(Y)
  peaks, _ = scipy.signal.find_peaks(Y)
  # plot vline at peak
  out_str = f"{i+1}_cage has {len(peaks)} peaks: "
  for peak in peaks:
    out_str += f"{X[peak] * 1.0e-6:.4g}, "
    if plot:
      plt.plot(X[peak+1], lim_y[1] * 1.5**(i+1), color=color_list[c], marker=10, ms=10)
  print(out_str[:-2] + " MHz")

def plot_stat_dft(df, vec="x", se=None, flim=25, i_list=[0]):
  n = re.search(r'(\d+)cages', this_stat_file.value)[1]
  n = int(n)
  # n = (df.shape[1]-1) // 6 - 1
  if 0 in i_list:
    n_list = range(n)
  else:
    n_list = [int(j - 1) for j in i_list]
  cmap = plt.colormaps.get_cmap(color_map)
  if len(n_list) == 1:
    color_list = ["black"]
  else:
    color_list = [cmap(i/np.max(n_list)) for i in n_list]
  n_lim = (df.shape[0] * flim) // 100
  X = df["freq"]
  X = X[1:n_lim]
  for c, i in enumerate(n_list):
    Y, Y_err = extract_direc(df, i, direction=vec)
    if se != None:
      Y_err = Y_err / np.sqrt(se)
    Y_under = Y - Y_err
    Y_under[Y_under <= 0] = np.mean(Y) * 1.0e-10
    Y_over = Y + Y_err
    Y = Y[1:n_lim]
    Y_under = Y_under[1:n_lim]
    Y_over = Y_over[1:n_lim]
    plt.fill_between(X, Y_under, Y_over, color=color_list[c], alpha=0.3)
    plt.plot(X, Y, "-", label=f"{i+1}_{vec}_ave.", color=color_list[c])

stat_path = os.path.join("Statistic", this_stat_file.value)
df = pd.read_csv(stat_path)
lim_y = [df.iloc[:, 1:].max().max() * 1.1, df.iloc[:, 1:].min().min() * 0.9]

plt.figure(figsize=(8, 6))

if 0 in j_list:
  s_list = range(n)
else:
  s_list = [int(j - 1) for j in j_list]
for c, i in enumerate(s_list):
  plot_peak(df, flim=freq_range, vec=direction, i=i, c=c, plot=Plot_peak)
plot_stat_dft(df, flim=freq_range,vec=direction, i_list=i_list, se=SEN)

plt.xscale('log')
plt.yscale('log')
set_xlim_df(df, flim=freq_range)
plt.ylim(lim_y[1], lim_y[0])
plt.xlabel("Frequency [Hz]", fontsize=14)
plt.ylabel(direction + "-Amplitude [m]", fontsize=14)
plt.legend()

# plt.savefig("DFT.png", dpi=300)

plt.show()

# Other Tool

In [ ]:
#@title DFT visualized { display-mode: "form"}

def plot_dft(result, i, n):
  signal_x = result.y[i]
  signal_y = result.y[i+n]
  signal_z = result.y[i+2*n]
  DFT_x = np.fft.fft(signal_x)
  DFT_y = np.fft.fft(signal_y)
  DFT_z = np.fft.fft(signal_z)

  freq = np.fft.fftfreq(frames, d=(t_span[1] / frames))
  Power_x = np.abs(DFT_x) / frames * 2
  Power_y = np.abs(DFT_y) / frames * 2
  Power_z = np.abs(DFT_z) / frames * 2
  max_Power_x = np.max(Power_x[1:(frames // 2)])
  max_Power_y = np.max(Power_y[1:(frames // 2)])
  max_Power_z = np.max(Power_y[1:(frames // 2)])
  max_Power = max(max_Power_x, max_Power_y, max_Power_z)
  harmonic_f = np.sqrt(k/Mass) / (2*np.pi)

  plt.subplot(n, 2, 2*i+1)
  plt.title(f"Motion [i = {i+1}]")
  plt.plot(result.t, signal_x - np.mean(signal_x), label="x")
  plt.plot(result.t, signal_y - np.mean(signal_y), label="y")
  plt.plot(result.t, signal_z - np.mean(signal_z), label="z")
  # plt.plot(sol.t, re_signal, linestyle="--", label="Reconstructed")
  plt.legend()
  plt.xlabel('t [s]')
  plt.ylabel('r_i [m]')
  plt.xlim(*t_span)

  plt.subplot(n, 2, 2*i+2)
  plt.stem(freq[1:(frames // 2)], Power_x[1:(frames // 2)], linefmt="C0", markerfmt=" ", basefmt="-C0", label="x")
  plt.stem(freq[1:(frames // 2)], Power_y[1:(frames // 2)], linefmt="--C1", markerfmt=" ", basefmt="-C1", label="y")
  plt.stem(freq[1:(frames // 2)], Power_z[1:(frames // 2)], linefmt=":C2", markerfmt=" ", basefmt="-C2", label="z")
  plt.axvline(x=harmonic_f, color='r', linestyle='--', label=f"$f_c$: {harmonic_f/1.0e3:.3g} kHz")
  plt.title(f"Magnitude Spectrum [i = {i+1}]")
  plt.xlabel("Frequency [Hz]")
  plt.ylabel("Amplitude [m]")
  plt.xlim(left=frames, right=frames/t_span[1]/10)
  plt.ylim(bottom=0, top=max_Power*1.1)
  plt.xscale('log')
  plt.legend()


plt.figure(figsize=(12, 5*n))
plt.subplots_adjust(hspace=0.3)

for i in range(n):
  plot_dft(sol, i, n)

# plt.savefig("DFT.png", dpi=300)
plt.show()

In [ ]:
#@title **Select** result for DFT { display-mode: "form"}
files = os.listdir("N-simulations")
result_files = [f for f in files if os.path.splitext(f)[1] == '.csv']
result_files = np.sort(result_files)

print(f"Number of result files: {len(result_files)}")

# Drop down list
this_result_file = widgets.Dropdown(
    options=result_files,
    value=result_files[0],
    description='Select result:',
    disabled=False,
    layout={'width': '50%', 'height': '40px'},
    style={'description_width': 'initial'},
)

display(this_result_file)

In [ ]:
#@title DFT for **Selected** result

def i_dft(df, i):
  signal_x = df.iloc[:, i+1].to_numpy()
  signal_y = df.iloc[:, i+2].to_numpy()
  signal_z = df.iloc[:, i+3].to_numpy()
  n_sample = len(signal_x)
  DFT_x = np.fft.fft(signal_x)
  DFT_y = np.fft.fft(signal_y)
  DFT_z = np.fft.fft(signal_z)

  DFT_x = DFT_x[:(n_sample//2)] / n_sample * 2
  DFT_y = DFT_y[:(n_sample//2)] / n_sample * 2
  DFT_z = DFT_z[:(n_sample//2)] / n_sample * 2
  DFT = np.array([DFT_x, DFT_y, DFT_z])
  DFT = np.abs(DFT)
  return DFT

def N_dft(result, n=None):
  if n == None:
    file_name = os.path.basename(result)
    match = re.search(r'x(\d+)cages', file_name)
    if match:
      n = int(match.group(1))
    else:
      print("The number of nanocages is not found from file name.\nPlease set argument n.")
      return None
  df = pd.read_csv(result)
  N = df.shape[1] // (n*3)
  DFT_x, DFT_y, DFT_z = [], [], []
  for j in range(n):
    DFT_ix, DFT_iy, DFT_iz = [], [], []
    for i in range(N):
      DFT = i_dft(df, i*3*n + j*3)
      DFT_ix.append(DFT[0])
      DFT_iy.append(DFT[1])
      DFT_iz.append(DFT[2])
    DFT_x.append(DFT_ix)
    DFT_y.append(DFT_iy)
    DFT_z.append(DFT_iz)
  return np.array([DFT_x, DFT_y, DFT_z])

def read_time(path):
  df = pd.read_csv(path)
  time = df.iloc[:, 0].to_numpy()
  time_span = time[-1] - time[0]
  return time_span

def stat_dft(dft, time):
  dim = dft.shape
  n_sample = dim[3] * 2
  f_sample = time / n_sample
  freq = np.fft.fftfreq(n_sample, d=f_sample)
  freq = freq[0:(n_sample//2)]
  out_table = pd.DataFrame(np.zeros((len(freq), 1 + dim[1]*6)))
  column_name_base = ["mean_x_", "std_x_", "mean_y_", "std_y_", "mean_z_", "std_z_"]
  column_names = ["freq"]
  for j in range(dim[1]):
    this_column_name = [s + str(j+1) for s in column_name_base]
    column_names = column_names + this_column_name
  out_table.columns = column_names
  out_table["freq"] = freq
  for j in range(dim[1]):
    this_dft = dft[:, j, :]
    out_table[f"mean_x_{j+1}"] = np.mean(this_dft[0], axis=0)
    out_table[f"std_x_{j+1}"] = np.std(this_dft[0], axis=0, ddof=1)
    out_table[f"mean_y_{j+1}"] = np.mean(this_dft[1], axis=0)
    out_table[f"std_y_{j+1}"] = np.std(this_dft[1], axis=0, ddof=1)
    out_table[f"mean_z_{j+1}"] = np.mean(this_dft[2], axis=0)
    out_table[f"std_z_{j+1}"] = np.std(this_dft[2], axis=0, ddof=1)
  return out_table

stat_dir = "Statistic"
dft_dir = "DFT-simulations"
os.makedirs(stat_dir, exist_ok=True)
os.makedirs(dft_dir, exist_ok=True)

this_path = os.path.join("N-simulations", this_result_file.value)
DFT_table = N_dft(this_path)
DFT_stat = stat_dft(DFT_table, read_time(this_path))
DFT_stat.head()

DFT_stat_path = os.path.join(stat_dir, f"{os.path.splitext(this_result_file.value)[0]}_stat.csv")
DFT_stat.to_csv(DFT_stat_path, index=False, float_format='%.6g')
DFT_table_path = os.path.join(dft_dir, f"{os.path.splitext(this_result_file.value)[0]}_DFT.npy")
np.save(DFT_table_path, DFT_table)

print(f"Save as DFT: {DFT_table_path}")
print(f"Save as Statistic: {DFT_stat_path}")
file_size_bytes_dft = os.path.getsize(DFT_table_path)
print(f"File size DFT: {format_bytes(file_size_bytes_dft)}")
file_size_bytes_stat = os.path.getsize(DFT_stat_path)
print(f"File size Statistic: {format_bytes(file_size_bytes_stat)}")

In [ ]:
#@title Save all results

!zip -r N-simulations.zip N-simulations
!zip -r DFT-simulations.zip DFT-simulations
!zip -r Statistic.zip Statistic

In [ ]:
#@title **!!Clear!!** all results in `N-siulations/`

import os
import shutil

def delete_files(dir_name):
  # Check if the folder exists
  if os.path.exists(dir_name):
      # Iterate over all items in the folder
      for item_name in os.listdir(dir_name):
          item_path = os.path.join(dir_name, item_name)
          try:
              # If it's a file, remove it
              if os.path.isfile(item_path):
                  os.remove(item_path)
                  print(f"Removed file: {item_path}")
              # If it's a directory, remove it and all its contents
              elif os.path.isdir(item_path):
                  shutil.rmtree(item_path)
                  print(f"Removed directory and its contents: {item_path}")
          except Exception as e:
              print(f"Error removing {item_path}: {e}")
      print(f"All contents in '{dir_name}' cleared.")
  else:
      print(f"Folder '{dir_name}' does not exist.")

delete_files("N-simulations")
delete_files("DFT-simulations")
delete_files("Statistic")